In [ ]:
from scipy.integrate import solve_ivp
import numpy as np

# Input data
MD_US = np.array([6750, 7050])
inc = np.array([30.9, 43.2]) # in degrees
azi = np.array([78.2, 100.6])  # in degrees
bw_pipe = 16  # Assuming constant value
friction_factor = 0.3
F_s = 25000  # Initial force at the end

# Convert degrees to radians
inc_rad = np.radians(inc)
azi_rad = np.radians(azi)

# Minimum Curvature Method
betta = np.arccos(np.sin(inc_rad[:-1])*np.sin(inc_rad[1:])*np.cos(np.diff(azi_rad)) + np.cos(inc_rad[:-1])*np.cos(inc_rad[1:]))
betta = np.nan_to_num(betta, nan=0.0)
RF = np.where(betta == 0, np.diff(MD_US)/2, np.diff(MD_US)/betta*np.tan(betta/2))
DLS = betta/np.diff(MD_US)

# B, T, and inc_ave calculations
B = np.diff(inc)/np.diff(MD_US)
T = np.diff(azi)/np.diff(MD_US)
inc_ave = (inc_rad[:-1] + inc_rad[1:]) / 2

# Tension, Normal force, and Side force calculations optimized
t_z = np.cos(inc_ave)
n_z = -1 / np.degrees(DLS) * np.sin(inc_ave) * B
b_z = 1 / np.degrees(DLS) * np.sin(inc_ave)**2 * T

# Function to compute dF/ds, note that the sign is negative because we integrate backwards
def dFds(s, F):
    w_c = ((F*DLS + bw_pipe*n_z)**2 + (bw_pipe*b_z)**2)**0.5
    return -(bw_pipe*t_z + friction_factor*w_c)

# Solution
solution = solve_ivp(dFds, np.flip(MD_US), [F_s], method='RK45', max_step=1)
solution.y



In [ ]:
from scipy.integrate import solve_ivp
import numpy as np

# Input data
MD = np.array([6750, 7050])
inc = np.array([30.9, 43.2]) # in degrees
azi = np.array([78.2, 100.6])  # in degrees
bw_pipe = 16  # Assuming constant value
friction_factor = 0.3
F_s = 25000  # Initial force at the end

# Convert degrees to radians
inc_rad = np.radians(inc)
azi_rad = np.radians(azi)

# Minimum Curvature Method
cos_delta_azi = np.cos(np.diff(azi_rad))
md_diff = np.diff(MD)

betta = np.arccos(np.sin(inc_rad[:-1])*np.sin(inc_rad[1:])*cos_delta_azi + np.cos(inc_rad[:-1])*np.cos(inc_rad[1:]))
betta = np.nan_to_num(betta, nan=0.0)
RF = np.where(betta == 0, md_diff/2, md_diff/betta*np.tan(betta/2))
DLS = betta/md_diff

# Displacement calculations
del_x = RF*(np.sin(inc_rad[:-1])*np.cos(azi_rad[:-1]) + np.sin(inc_rad[1:])*np.cos(azi_rad[1:]))
del_y = RF*(np.sin(inc_rad[:-1])*np.sin(azi_rad[:-1]) + np.sin(inc_rad[1:])*np.sin(azi_rad[1:]))
del_z = RF*(np.cos(inc_rad[:-1]) + np.cos(inc_rad[1:]))
MD_s = (MD[:-1] + MD[1:])/2

# Tension, Normal force, and Side force calculations optimized
t_z_min_val = np.cos(inc_rad[:-1]) * np.cos(DLS * (MD_s - MD[:-1])) + \
    ((np.cos(inc_rad[1:]) - np.cos(inc_rad[:-1]) * np.cos(betta)) / np.sin(betta)) * np.sin(DLS * (MD_s - MD[:-1]))
t_z = np.where(np.isnan(t_z_min_val), np.cos((inc_rad[1:] + inc_rad[:-1]) / 2), t_z_min_val)
n_z = -np.cos(inc_rad[:-1]) * np.sin(DLS * (MD_s - MD[:-1])) + \
    ((np.cos(inc_rad[1:]) - np.cos(inc_rad[:-1]) * np.cos(betta)) / np.sin(betta)) * np.cos(DLS * (MD_s - MD[:-1]))
n_z = np.nan_to_num(n_z)
b_z = np.sin(inc_rad[:-1]) * np.sin(inc_rad[1:]) * np.sin(azi_rad[1:] - azi_rad[:-1]) / np.sin(betta)
b_z = np.nan_to_num(b_z)

# Function to compute dF/ds, note that the sign is negative because we integrate backwards
def dFds(s, F):
    w_c = ((F*DLS + bw_pipe*n_z)**2 + (bw_pipe*b_z)**2)**0.5
    return -(bw_pipe*t_z + friction_factor*w_c)

# Solution
solution = solve_ivp(dFds, np.flip(MD), [F_s], method='RK45', max_step=1)
solution.y
